In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
raw_data_dir = "/data/put_data/cmchang/gynecology/raw_data/"
process_data_dir = "/data/put_data/cmchang/gynecology/data/"

In [4]:
dx = pd.read_csv(os.path.join(raw_data_dir, 'data_20180911.csv'))
dy = pd.read_csv(os.path.join(raw_data_dir, 'interpretation_new_180927.csv'))

print("dx.shape = %s, dy.shape = %s" % (dx.shape, dy.shape))

dx.shape = (128760, 68), dy.shape = (213, 12)


In [5]:
dx.head()

,counter,machine_type,type,bed,mins_diff,datetime,date,time,_00,_01,...,_50,_51,_52,_53,_54,_55,_56,_57,_58,_59
0,1,1,1,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,46,...,24,22,24,22,21,22,20,20,20,22
1,2,1,2,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,138,...,141,140,141,140,140,141,140,138,139,139
2,3,1,3,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,01,2025497,2011-04-07 14:17:00,2011-04-07,14:17:00,20,20,...,22,20,0,21,21,21,22,23,24,25
4,5,1,2,01,2025497,2011-04-07 14:17:00,2011-04-07,14:17:00,138,138,...,140,139,0,138,137,137,138,138,138,138


In [6]:
dy.head()

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,0,0
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,2,1
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,2,1
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,2,1
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,2,1


#### remove X but without Y

In [7]:
selector = [k in set(dy.date) for k in dx.date]

dx = dx[selector]

print("dx.shape = %s, dy.shape = %s" % (dx.shape, dy.shape))

dx.shape = (62380, 68), dy.shape = (213, 12)


#### create key

In [8]:
dx['seconds'] = (pd.to_datetime(dx.datetime) - pd.to_datetime(dx.date)).dt.seconds
dx['key'] = dx.apply(lambda row: str(row['date']) +' '+ str(row['seconds']), axis=1)

In [9]:
dy['datetime'] = dy.date + ' ' + dy.start
dy['seconds'] = (pd.to_datetime(dy.datetime) - pd.to_datetime(dy.date)).dt.seconds
dy['key'] = dy.apply(lambda row: str(row['date']) +' '+ str(row['seconds']), axis=1)

#### remove Y but without X

In [10]:
selector = [k in set(dx.key) for k in dy.key]

dy = dy[selector]

In [11]:
if set(dy.key) - set(dx.key):
    print('%i labels have no inputs.' % len(set(dy.key) - set(dx.key)))
    print(set(dy.key) - set(dx.key))
else:
    print('all labels have corresponding inputs.')


all labels have corresponding inputs.


In [12]:
# only have 8 minutes ... (?)
dy[dy.key=='2015-08-28 66420']

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management,datetime,seconds,key
134,324546,LDR1,2015-08-28,18:27,18:35,135,0,1,90.0,60.0,0,0,2015-08-28 18:27,66420,2015-08-28 66420


In [13]:
dy.drop(dy.index[list(dy.key).index('2015-08-28 66420')], inplace=True)

In [14]:
dy.sort_values('key', inplace=True)

In [15]:
dy.reset_index(inplace=True)
dy.drop('index', axis=1, inplace=True)

In [16]:
dy.head()

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management,datetime,seconds,key
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,0,0,2013-01-15 17:20,62400,2013-01-15 62400
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,2,1,2013-01-15 17:54,64440,2013-01-15 64440
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,2,1,2013-01-15 18:15,65700,2013-01-15 65700
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,2,1,2013-01-15 18:29,66540,2013-01-15 66540
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,2,1,2013-01-15 18:39,67140,2013-01-15 67140


In [18]:
mm_dx = dx[dx.type==1]
bb_dx = dx[dx.type==2]

In [19]:
mm_dx.head(3)

,counter,machine_type,type,bed,mins_diff,datetime,date,time,_00,_01,...,_52,_53,_54,_55,_56,_57,_58,_59,seconds,key
2913,2914,1,1,LDR2,2960152,2013-01-15 15:52:00,2013-01-15,15:52:00,20,20,...,20,20,20,20,20,20,20,20,57120,2013-01-15 57120
2914,2915,1,1,LDR2,2960153,2013-01-15 15:53:00,2013-01-15,15:53:00,20,20,...,21,21,21,21,21,21,21,21,57180,2013-01-15 57180
2915,2916,1,1,LDR2,2960154,2013-01-15 15:54:00,2013-01-15,15:54:00,21,21,...,21,21,21,21,21,21,21,21,57240,2013-01-15 57240


In [20]:
mm_val_idx = list()
for k in mm_dx.columns:
    if len(k) == 3 and '_' in k:
        mm_val_idx.append(k)
        print(k, end='\t')

_00	_01	_02	_03	_04	_05	_06	_07	_08	_09	_10	_11	_12	_13	_14	_15	_16	_17	_18	_19	_20	_21	_22	_23	_24	_25	_26	_27	_28	_29	_30	_31	_32	_33	_34	_35	_36	_37	_38	_39	_40	_41	_42	_43	_44	_45	_46	_47	_48	_49	_50	_51	_52	_53	_54	_55	_56	_57	_58	_59	

In [21]:
bb_val_idx = list()
for k in bb_dx.columns:
    if len(k) == 3 and '_' in k:
        bb_val_idx.append(k)
        print(k, end='\t')

_00	_01	_02	_03	_04	_05	_06	_07	_08	_09	_10	_11	_12	_13	_14	_15	_16	_17	_18	_19	_20	_21	_22	_23	_24	_25	_26	_27	_28	_29	_30	_31	_32	_33	_34	_35	_36	_37	_38	_39	_40	_41	_42	_43	_44	_45	_46	_47	_48	_49	_50	_51	_52	_53	_54	_55	_56	_57	_58	_59	

In [22]:
dy.key

0      2013-01-15 62400
1      2013-01-15 64440
2      2013-01-15 65700
3      2013-01-15 66540
4      2013-01-15 67140
5      2013-01-15 69540
6      2013-01-15 71820
7      2013-01-15 73140
8      2013-01-15 74940
9      2013-01-15 76140
10     2013-01-15 77340
11     2013-01-18 30840
12     2013-01-18 51240
13     2013-01-28 11580
14     2013-01-28 19380
15     2013-02-15 54180
16     2013-02-21 27480
17       2013-03-03 960
18     2013-03-06 76260
19     2013-04-17 63840
20     2013-05-01 55980
21     2013-05-01 58980
22     2013-05-01 63180
23     2013-05-14 61740
24     2013-05-19 76980
25     2013-05-20 33120
26     2013-05-25 37380
27     2013-05-25 48180
28     2013-05-27 47580
29     2013-05-27 47580
             ...       
177    2017-05-07 79740
178    2017-05-13 16740
179    2017-05-13 22140
180    2017-05-13 29340
181    2017-06-10 46320
182    2017-08-16 69720
183    2017-08-23 55140
184    2017-09-12 59280
185    2017-09-12 81300
186    2017-09-13 15360
187     2017-09-

In [23]:
M_list = list()
for k in dy.key:
    try:
        i = list(mm_dx.key).index(k)

        tmp = mm_dx[i:(i+10)]
        checkpoint = tmp.seconds.tolist()
        if (checkpoint[-1] - checkpoint[0]) == 60*9:
            M_list.append(np.array(mm_dx[i:(i+10)][mm_val_idx]).reshape(-1))
            continue
        else:
            print('%s has problem' % k)
    except ValueError as err:
        print(err)

In [24]:
B_list = list()
for k in dy.key:
    try:
        i = list(bb_dx.key).index(k)
        tmp = bb_dx[i:(i+10)]
        checkpoint = tmp.seconds.tolist()
        if (checkpoint[-1] - checkpoint[0]) == 60*9:
            B_list.append(np.array(bb_dx[i:(i+10)][bb_val_idx]).reshape(-1))
            continue
        else:
            print('%s has problem' % k)
    except ValueError as err:
        print(err)

In [25]:
M = pd.DataFrame(M_list, columns=['m-{0:0>3}'.format(i) for i in range(len(M_list[0]))])

B = pd.DataFrame(B_list, columns=['b-{0:0>3}'.format(i) for i in range(len(B_list[0]))])

In [26]:
B.shape

(207, 600)

In [27]:
M.shape

(207, 600)

In [28]:
dy.shape

(207, 15)

In [29]:
dM = pd.concat([dy, M], axis=1)

dM.to_csv(os.path.join(process_data_dir, 'data_M.csv'), index=False)

In [30]:
dB = pd.concat([dy, B], axis=1)

dB.to_csv(os.path.join(process_data_dir, 'data_B.csv'), index=False)

In [31]:
df = pd.concat([dy, M], axis=1)
df = pd.concat([df, B], axis=1)

df.to_csv(os.path.join(process_data_dir, 'data_merged.csv'), index=False)